In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler

In [2]:
dataset1 = pd.read_csv('data/dataset1.csv')
dataset1.drop_duplicates(inplace=True)
dataset1.ix[:5]

,user_id,discount_rate,distance,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,total_sales,sales_use_coupon,...,day_gap_after,is_weekend,weekday1,weekday2,weekday3,weekday4,weekday5,weekday6,weekday7,label
0,1832624,0.900000,0,29,-62,200,20,1,14962.0,2168.0,...,0.0,0,0,0,0,0,1,0,0,0
1,163606,0.850000,10,21,-70,200,30,1,787.0,105.0,...,0.0,0,0,0,0,1,0,0,0,0
2,4061024,0.900000,10,26,-65,200,20,1,14962.0,2168.0,...,0.0,0,0,1,0,0,0,0,0,0
3,106443,0.833333,0,29,-62,30,5,1,6553.0,408.0,...,0.0,0,0,0,0,0,1,0,0,0
4,114747,0.850000,9,26,-65,200,30,1,787.0,105.0,...,0.0,0,0,1,0,0,0,0,0,0
5,114747,0.850000,9,26,-65,200,30,1,787.0,105.0,...,0.0,0,0,1,0,0,0,0,0,0


In [3]:
dataset2 = pd.read_csv('data/dataset2.csv')
dataset2.drop_duplicates(inplace=True)
dataset2.ix[:5]

,user_id,discount_rate,distance,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,total_sales,sales_use_coupon,...,day_gap_after,is_weekend,weekday1,weekday2,weekday3,weekday4,weekday5,weekday6,weekday7,label
0,1439408,0.866667,1,28,-33,150,20,1,657.0,26.0,...,0.0,1,0,0,0,0,0,1,0,0
1,1439408,0.950000,0,13,-17,20,1,1,14.0,3.0,...,0.0,0,1,0,0,0,0,0,0,0
2,1439408,0.950000,0,16,-45,20,1,1,14.0,3.0,...,0.0,0,1,0,0,0,0,0,0,-1
3,2029232,0.833333,0,30,-31,30,5,1,6307.0,643.0,...,0.0,0,1,0,0,0,0,0,0,0
4,2029232,0.950000,0,19,-42,20,1,1,5.0,0.0,...,0.0,0,0,0,0,1,0,0,0,0
5,2747744,0.800000,0,6,-24,50,10,1,7471.0,390.0,...,0.0,0,1,0,0,0,0,0,0,0


In [4]:
dataset3 = pd.read_csv('data/dataset3.csv')
dataset3.drop_duplicates(inplace=True)
dataset3.ix[:5]

,user_id,coupon_id,discount_rate,distance,date_received,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,...,day_gap_before,day_gap_after,is_weekend,weekday1,weekday2,weekday3,weekday4,weekday5,weekday6,weekday7
0,4129537,9983,0.833333,1,20160712,12,12,30,5,1,...,0.0,0.0,0,0,1,0,0,0,0,0
1,6949378,3429,0.833333,0,20160706,6,6,30,5,1,...,0.0,0.0,0,0,0,1,0,0,0,0
2,2166529,6928,0.900000,5,20160727,27,27,200,20,1,...,0.0,0.0,0,0,0,1,0,0,0,0
3,2166529,1808,0.900000,5,20160727,27,27,100,10,1,...,0.0,0.0,0,0,0,1,0,0,0,0
4,6172162,6500,0.966667,2,20160708,8,8,30,1,1,...,0.0,0.0,0,0,0,0,0,1,0,0
5,4005121,9983,0.833333,0,20160706,6,6,30,5,1,...,0.0,0.0,0,0,0,1,0,0,0,0


In [5]:
dataset12 = pd.concat([dataset1,dataset2],axis=0)

In [6]:
dataset1_y = dataset1.label
dataset2_y = dataset2.label
dataset12_y = dataset12.label
# 'day_gap_before','day_gap_after'
dataset1_x = dataset1.drop(['user_id','label','day_gap_before','day_gap_after'],axis=1)  
dataset2_x = dataset2.drop(['user_id','label','day_gap_before','day_gap_after'],axis=1)
dataset12_x = dataset12.drop(['user_id','label','day_gap_before','day_gap_after'],axis=1)
dataset3_x = dataset3.drop(['user_id','coupon_id','date_received','day_gap_before','day_gap_after'],axis=1)

dataset3_preds = dataset3[['user_id','coupon_id','date_received']]

In [7]:
# read in data
dataset1 = xgb.DMatrix(dataset1_x,label=dataset1_y)
dataset2 = xgb.DMatrix(dataset2_x,label=dataset2_y)
dataset3 = xgb.DMatrix(dataset3_x)

dataset12 = xgb.DMatrix(dataset12_x,label=dataset12_y)

In [8]:
params={'booster':'gbtree',
	    'objective': 'rank:pairwise',
	    'eval_metric':'auc',
	    'gamma':0.1,
	    'min_child_weight':1.1,
	    'max_depth':8,
	    'lambda':10,
	    'subsample':0.7,
	    'colsample_bytree':0.7,
	    'colsample_bylevel':0.7,
	    'eta': 0.01, 
	    'tree_method':'exact',
	    'seed':0,
	    'nthread':12
	    }
watchlist = [(dataset12,'train')]
model = xgb.train(params,dataset12,num_boost_round=3500,evals=watchlist)

[11:34:02] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[0]	train-auc:0.820028
[11:34:02] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[1]	train-auc:0.845253
[11:34:02] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[2]	train-auc:0.84448
[11:34:02] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[3]	train-auc:0.848305
[11:34:02] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[4]	train-auc:0.849165
[11:34:03] d:\build\xgboost\xgboost-0.80.git\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[5]	train-

In [9]:
# make prediction 预测
dataset3_preds['label'] = model.predict(dataset3)
dataset3_preds.label = MinMaxScaler().fit_transform(dataset3_preds.label.reshape(-1, 1))
dataset3_preds.sort_values(by=['coupon_id','label'],inplace=True)
dataset3_preds.to_csv("xgb_preds.csv",index=None,header=None)
dataset3_preds.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  app.launch_new_instance()


,user_id,coupon_id,date_received,label
28711,2775265,3,20160723,0.220300
179501,1302487,3,20160709,0.220397
11546,5636038,3,20160723,0.221414
175054,5244608,3,20160720,0.227408
59378,1647633,3,20160710,0.227738
181064,1018354,3,20160724,0.228347
175055,5244608,3,20160720,0.228923
232798,5712798,3,20160709,0.231350
275509,2735138,3,20160701,0.249991
271883,1368563,3,20160722,0.255127


In [10]:
#save feature score
feature_score = model.get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
fs = []
for (key,value) in feature_score:
    fs.append("{0},{1}\n".format(key,value))
    
with open('xgb_feature_score.csv','w') as f:
    f.writelines("feature,score\n")
    f.writelines(fs)